In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import mglearn
import os

# cross varidation 交差検証  
- モデルを作る上で重要な指標の一つに汎化性能がある。なぜなら、モデルを作るモチベーションは何かを予測したい、であり、予測するデータは未知なデータなので。
- モデルを作るために準備した特長量と、それに適用する予測器のアルゴリズムがどの程度汎化性能を持っているかどうかは、testデータに対するscoreで見ていたが、train testの分け方はランダムなので、1回の思考でいいのか？という観点。
## k-fold k分割, stratified k-fold 層化k分割

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

In [ ]:
iris=load_iris()
iris.keys()

In [ ]:
logreg=LogisticRegression(max_iter=2000)
scores=cross_val_score(estimator=logreg,X=iris.data,y=iris.target,cv=3)
scores

In [ ]:
print(f'average of cross val score : {np.average(scores):.3f}')

- クロスバリデーションはモデルを返さない。  
モデルを作るための手法ではなく、与えたデータセットに対して指定したアルゴリズム（今回で言えばロジスティック回帰）がどの程度汎化できるかの目安を見るだけ。  
今回の例で言えば、irisのデータセットは、ロジスティック回帰で大体97%の割合で品種を当てられそうだ、ということがわかった

* 単純なk分割交差だと、データセットの並び順によって分割されたデータセット内の目的変数にばらつきが生じる可能性がある  
* クラス分割の場合は、目的変数ごとに等分する層化k分割交差検証をした方がいい  
* sklearnはestimaterがクラス分類器の場合は層化、回帰器の場合は通常のk分割をデフフォルトで採用してくれる。
* 分割器をimportしてcvパラメータに明示的に指定することも可能

In [ ]:
mglearn.plots.plot_stratified_cross_validation()

In [ ]:
from sklearn.model_selection import KFold
# irisデータセットに層化しない3分割交差検証をすると、訓練したデータセット内にテストの目的変数がないため精度はゼロ
logreg=LogisticRegression(max_iter=2000)
scores=cross_val_score(estimator=logreg,X=iris.data,y=iris.target,cv=KFold(n_splits=3))
scores

In [ ]:
# 単純分割でもデータセットをすれば確率的にばらける
scores=cross_val_score(estimator=logreg,X=iris.data,y=iris.target,cv=KFold(n_splits=3,shuffle=True,random_state=0))
scores

In [ ]:
from sklearn.model_selection import StratifiedKFold
# 層化の方が安定
scores=cross_val_score(estimator=logreg,X=iris.data,y=iris.target,cv=StratifiedKFold(n_splits=3))
scores

## 1つ抜き交差検証
- データセットから1つをテストデータとして、それ以外のデータで訓練したモデルを使って残りの一つを当てる、というのを総当たりで繰り返して汎化性能を求める

In [ ]:
from sklearn.model_selection import LeaveOneOut
scores=cross_val_score(estimator=logreg,X=iris.data,y=iris.target,cv=LeaveOneOut())
print(f'number of cv iter : {len(scores)}')
print(f'avg of scores : {scores.mean()}')

## shuffle split シャッフル分割
- データセットからn_tr個 or f_tr%の訓練データと、それに重複しないn_te個 or f_te%のテストデータを取り出してscoreを求めることをn_split回実行する、というやり方

In [ ]:
mglearn.plots.plot_shuffle_split()

In [ ]:
from sklearn.model_selection import ShuffleSplit
shuffle=ShuffleSplit(train_size=.5,test_size=.5,n_splits=10,random_state=42) # 50％ずつ訓練とテストに分けてscore算出を10回繰り返す
scores=cross_val_score(estimator=logreg,X=iris.data,y=iris.target,cv=shuffle)
print(f'number of split : {len(scores)}')
print(f'avg of scores : {scores.mean()}')
# 層化シャッフル分割もある。

## グループ付交差検証
- 例えば100人から10枚ずつ集めた写真をデータセットとして、表情から感情を当てる予測器を作るとき、予測器に求められる汎化性能は、「訓練データに入っていない人の新しい表情から感情を当てること」。一方で、予測の視点で言えば、訓練した人の新しい表情から感情を当てる方が簡単。  
このため、交差検証において、訓練データとテストデータに同じ人の画像が入っていると、精度が高くでがち。  
これを防ぐために、同じ人、というグループの概念を加味して分割する。
  - 具体的には、同じグループのデータは訓練とテストのいずれかにまとまるように分割する
    - 現実世界でこういうケースは多く、同じ患者から得られた複数サンプルをもとにモデルを作る（汎化性能は未知の人の予測）とか、同じ人から得られた発話をもとにモデルを作る、とか

In [ ]:
mglearn.plots.plot_group_kfold()

In [ ]:
from sklearn.model_selection import GroupKFold
from sklearn.datasets import make_blobs
X,y=make_blobs(n_samples=12,random_state=0)
groups = [0,0,0,1,1,1,1,2,2,3,3,3]
# サンプルサイズ12のデータは、4グループに属している、というデータの例

scores=cross_val_score(estimator=logreg,X=X,y=y,groups=groups,cv=GroupKFold(n_splits=3))
scores

# グリッドサーチ
- 汎化性能に影響する因子として、特長量、アルゴリズムの種類に加えて、ハイパーパラメータがある。
- パラメータのチューニングは大変。sklearnではそれをフォローするアルゴリズムとしてグリッドサーチを実装。
- パラメータの全ての組み合わせを試す方法
- RBFカーネル法を使ったSVMで例示

In [ ]:
mglearn.plots.plot_svm(log_C=-1,log_gamma=0)
"""
RBFカーネル法のSVMのおさらい
パラメータは2つ
gamma : データポイントが近い、とみなす距離を制御。
    小さい＝寛大な判定。そんな近くなくても近い
    大きい＝厳しい判定。ちゃんと近くないと近くない
C : 正則化パラメータ：1つ1つのデータポイントの重要度を制御。
    小さい＝強い正則化。一つひとつのデータポイントの重要度を下げる（精度は劣るが汎化に優れる）。
    大きい：弱い正則化。全てのデータポイントにしっかり向き合って予測（過学習のリスク
"""

In [ ]:
# 単純なグリッドサーチ
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(iris.data,iris.target,random_state=0)
best_score=0

for gamma in [.001,.01,.1,1,10,100]:
    for C in [.001,.01,.1,1,10,100]:
        svc=SVC(gamma=gamma,C=C).fit(X_train,y_train)
        score=svc.score(X_test,y_test)
        if score>best_score:
            best_score=score
            best_param={'gamma':gamma,'C':C}
print(best_score)
print(best_param)

- これはだめ
- trainでモデル構築してtestでパラメータチューニングしてしまったので、作ったモデルが未知のデータに対してどんな性能を示すかが検証できていない
- パラメータチューニングによってtestデータを過学習したモデルを作っている
  - データに都合いいモデルは組めたけど、未知のデータもこれに当てはまるかわからないよね？ -> trainでfittingしてtestで確認しましょう
  - 訓練データを使って、とっておいた残りのデータにも割と当てはまるモデルができた！
    - パラメータをいじってもっといい感じにしたいけど、残りのデータを全部使ってパラメータをいじったら、たまたまそのデータに都合いいだけのパラメータだったかもしれない  
    -> validationでtuningしてtestで確認しましょう

In [ ]:
mglearn.plots.plot_threefold_split()

In [ ]:
X_train_val,X_test,y_train_val,y_test=train_test_split(iris.data,iris.target,random_state=0)
X_train,X_val,y_train,y_val=train_test_split(X_train_val,y_train_val,random_state=1)
best_score=0



for gamma in [.001,.01,.1,1,10,100]:
    for C in [.001,.01,.1,1,10,100]:
        svc=SVC(gamma=gamma,C=C).fit(X_train,y_train) #訓練データで作ったモデルを
        score=svc.score(X_val,y_val) #バリデーションデータでチューニング
        if score>best_score:
            best_score=score
            best_param={'gamma':gamma,'C':C}
print(f'best score on validation : {best_score}')
print(f'best params : {best_param}')

svc=SVC(gamma=best_param['gamma'],C=best_param['C']).fit(X_train_val,y_train_val) #訓練データとバリデーションデータでモデルを再構築
score = svc.score(X_test,y_test)
print(f'score on test : {score}')

## 交差検証✖️グリッドサーチ
- データの分け方によって最適パラメータが変わったりするので、クロスバリデーションした方が確か

In [ ]:
for gamma in [.001,.01,.1,1,10,100]:
    for C in [.001,.01,.1,1,10,100]:
        svc=SVC(gamma=gamma,C=C) #訓練データで作ったモデルを
        scores=cross_val_score(estimator=svc,X=X_train_val,y=y_train_val,cv=5)
        score=scores.mean()
        if score>best_score:
            best_score=score
            best_param={'gamma':gamma,'C':C}
print(f'best score on validation : {best_score}')
print(f'best params : {best_param}')

svc=SVC(gamma=best_param['gamma'],C=best_param['C']).fit(X_train_val,y_train_val) #訓練データとバリデーションデータでモデルを再構築
score = svc.score(X_test,y_test)
print(f'score on test : {score}')

In [ ]:
mglearn.plots.plot_cross_val_selection()

In [ ]:
mglearn.plots.plot_grid_search_overview()

- 広義では交差検証はデータセットに対するアルゴリズムの汎化性能を評価する手法だが、現実的にはパラメータチューニングの際のグリッドサーチに交差検証を使うケースがほとんどなので、一般に交差検証＝交差検証を使ったグリッドサーチによるパラメータチューニングという意味で使われることが多い  
- 交差検証をする場合は、train-testに分けて、train内で交差検証をすれば良いので、validationデータを明示的に分けなくてOK  
- sklearnでは交差検証を用いたグリッドサーチによるパラメータチューニングのためにGridSearchCVクラスがある

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'gamma':[.001,.01,.1,1,10,100],
    'C':[.001,.01,.1,1,10,100]
}
# GridSearchCVインスタンス作成
# 引数は分類器のアルゴリズム、アルゴリズムに応じたパラメータとテストしたい値の辞書、クロスバリデーションの方法（分割数）
grid_search=GridSearchCV(estimator=SVC(),param_grid=param_grid,cv=5)
# このインスタンスはfitするだけで自動的にクロスバリデーションを実施して、最適なパラメータで再構築したモデルを返す。

X_train,X_test,y_train,y_test=train_test_split(iris.data,iris.target,random_state=0)
grid_search.fit(X_train,y_train)

In [ ]:
print(f'best score on cross validation {grid_search.best_score_:.3f}')
print(f'best params {grid_search.best_params_}')
print(f'accuracy on test {grid_search.score(X_test,y_test):.3f}')

In [ ]:
print(f'best estimator\n{grid_search.best_estimator_}')

In [ ]:
cv_result=pd.DataFrame(grid_search.cv_results_).loc[:,'param_C':]

In [ ]:
for i,arg in enumerate(cv_result.columns):
    print(f'{i:<2}:{arg}')

In [ ]:
cv_result[cv_result.columns[[0,1,8,9,10]]].sort_values('rank_test_score',ascending=True)

In [ ]:
scores=np.array(cv_result.mean_test_score).reshape(6,6)

In [ ]:
result=cv_result.pivot(index='param_C',columns='param_gamma',values='mean_test_score').sort_index(ascending=False)

In [ ]:
result

In [ ]:
import seaborn as sns
sns.set()
sns.heatmap(result,annot=True)

- 今回はうまくいったが、パラメータの検証はばが小さいと全部ダメダメだったりすることも

In [ ]:
param_grid_liner = {
    'gamma':np.linspace(1,2,6),
    'C':np.linspace(1,2,6),
}
param_grid_one_log = {
    'gamma':np.logspace(-3,2,6),
    'C':np.linspace(1,2,6),
}
param_grid_log = {
    'gamma':np.logspace(-1,4,6),
    'C':np.logspace(-3,2,6),
}

fig,axes =plt.subplots(1,3,figsize=(15,5),tight_layout=True)

for ax,param_grid in zip(axes,[param_grid_liner,param_grid_one_log,param_grid_log]):
    grid_search=GridSearchCV(estimator=SVC(),param_grid=param_grid,cv=5)
    X_train,X_test,y_train,y_test=train_test_split(iris.data,iris.target,random_state=0)
    grid_search.fit(X_train,y_train)
    result=pd.DataFrame(grid_search.cv_results_)
    result=result.pivot(index='param_C',columns='param_gamma',values='mean_test_score').sort_index(ascending=False)
    sns.heatmap(result,annot=True,ax=ax,vmin=0,vmax=1)

- 総当たりだけでなく、組み合わせも指定できる、param_aがhogeの場合はparam_bとparam_c,param_aがfugaの場合はparam_dとparam_eみたいな

In [ ]:
param_grid=[
    {
        'kernel':['rbf'], # 辞書のvalueはリスト型の必要あり
        'gamma':[.001,.01,.1,1,10,100],
        'C':[.001,.01,.1,1,10,100]
    },
    {
        'kernel':['linear'],
        'C':[.001,.01,.1,1,10,100]
    }
]
grid_search=GridSearchCV(estimator=SVC(),param_grid=param_grid,cv=5)
X_train,X_test,y_train,y_test=train_test_split(iris.data,iris.target,random_state=0)
grid_search.fit(X_train,y_train)
# result=pd.DataFrame(grid_search.cv_results_)


In [ ]:
print(f'best score on cv : {grid_search.best_score_}')
print(f'best params : {grid_search.best_params_}')

* ここまでのクロスバリデーションでは、訓練データの中の訓練データ/検証データの偏りを防いでいた。  
複数のパラメータ条件を検証する際に、1つの条件検証内で何回もデータ分割を行う、という考え方。  
  - しかし、そのための初手に訓練データとテストデータを１発分けているが、ここで偏りがあったらどうする・・・？  
    - 初手のtrain&valid-testもクロスバリデーションして、それぞれのtraiｎ-validもクロスバリデーションすればOK！  
    たくさんのデータセットでそれぞれパラメータチューニングをするので、最適なパラメータを求めているのではなく、モデル、パラメータまで込み込みの汎化性能を示す  
    __データセットの評価の側面が大きい。このデータセットだったら、このアルゴリズムでパラメータチューニングを頑張ったら汎化性能が高いモデルが作れるね！ということがわかる__

In [ ]:
scores=cross_val_score(grid_search,X=iris.data,y=iris.target,cv=5,n_jobs=-1)
print(f'cv scores : {scores}')
print(f'mean cv scores : {scores.mean()}')

# ここからはモデルの評価の話  
- 元々のデータセットに偏りがあるとaccuracy(精度：テストサンプルのうちの正答率)が高くても、ちゃんと学習した優れたモデルといえないケースがある

In [ ]:
from sklearn.datasets import load_digits
digits = load_digits()
y = digits.target == 9

X_train,X_test,y_train,y_test = train_test_split(digits.data, y, random_state=0)

from sklearn.dummy import  DummyClassifier
dummy_majority=DummyClassifier(strategy='most_frequent').fit(X_train,y_train)
pred_most_freq=dummy_majority.predict(X_test)
# 訓練データの最頻クラスを返すだけの分類器
print(f'accuracy on test by dummy : {dummy_majority.score(X_test,y_test)}')

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(max_depth=3,random_state=42).fit(X_train,y_train)
pred_randomforest=rfc.predict(X_test)
print(f'accuracy on test by random forest : {rfc.score(X_test,y_test)}')

In [ ]:
from sklearn.dummy import DummyClassifier
dummy=DummyClassifier(strategy='prior').fit(X_train,y_train)
pred_dummy=dummy.predict(X_test)
print(f'dummy score : {dummy.score(X_test,y_test)}')

In [ ]:
from sklearn.linear_model import LogisticRegression
logreg=LogisticRegression(max_iter=2000).fit(X_train,y_train)
pred_logreg=logreg.predict(X_test)
print(f'logreg score : {logreg.score(X_test,y_test)}')

## 混合行列:confusion matrix

In [ ]:
from sklearn.metrics import  confusion_matrix
confusion = confusion_matrix(y_test,pred_logreg)
print(f'confusion matrix : \n {confusion}')

In [ ]:
mglearn.plots.plot_confusion_matrix_illustration()

In [ ]:
mglearn.plots.plot_binary_confusion_matrix()

In [ ]:
d={
    'most freq':pred_most_freq,
    'dummy':pred_dummy,
    'random forest':pred_randomforest,
    'logreg':pred_logreg
}

for k,v in d.items():
    confusion=confusion_matrix(y_test,v)
    print(f'{k}:\n{confusion}')

- __precision:適合率__ = モデルが陽性と判定したサンプルのうち本当に陽性だったサンプルの割合。的中率。モデルは陽性をどれだけ当てられたか。
- __recall:再現率__ = 実際の陽性のうち、モデルが正しく陽性と判定したサンプルの割合。感度。真の陽性をどれくらい取りこぼさなかったか。
- __F1スコア__ = precisionとrecallの調和平均。
$$F1 score = \frac{2}{\frac{1}{precision} + \frac{1}{recall}} = 2 \times \frac{precision + recall}{precision \times recall}$$

In [ ]:
from sklearn.metrics import f1_score
for k,v in d.items():
    f1=f1_score(y_test,v)
    print(f'{k:<20}:{f1:.2f}')

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(
    y_test,
    pred_logreg,
    target_names=['not nine','nine']
))

## 不確実性を考慮
- ２クラス分類の場合は、モデルの判定の不確実性を推定するための指標として以下の2つがある。
  - dicision function(決定関数) : レンジは決まっていないが0以上なら陽性、以下なら陰性
  - predict plobablity(確信度) : モデルが予測サンプルを陽性/陰性と判定した時のクラス分類の確率.

In [ ]:
mglearn.plots.plot_decision_threshold()

In [ ]:
from sklearn.datasets import make_blobs
X,y = make_blobs(n_samples=(400,50),cluster_std=[7.0,2],random_state=22)
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=0)
svc=SVC(gamma=.05).fit(X_train,y_train)

In [ ]:
plt.scatter(X[y==0,0],X[y==0,1],marker='o',label=0)
plt.scatter(X[y==1,0],X[y==1,1],marker='^',label=1)
plt.legend(loc='best')

In [ ]:
print(classification_report(y_test,svc.predict(X_test)))

- class 1の再現率を高めたい、という目線で調整するとしたら、適合率が低くても良い（偽陽性が多くてもいい）ので取りこぼしがないように（偽陰性を少なくしたい）ということで決定関数を0以下にずらす。  
そうするとモデルは1と判定しやすくなるので結果的に1の再現率は上がる

In [ ]:
y_pred_lower_threshold = svc.decision_function(X_test)>-.8

In [ ]:
print(classification_report(y_test,y_pred_lower_threshold))

- dicision_functionやpredict_probaをいじると適合率や再現率を操作することが可能だが、これらはトレードオフ。
  - 例えば閾値を上げて厳しく判定させると、慎重に判断する分モデルの適合率は上がるが、陽性と回答する数が減るので取りこぼしが多くなり再現率は下がる
  - 逆に閾値を下げて緩く判定させると、偽陰性が少なくなるので再現率は上がるが、大雑把に陽性だと判定するようになるので適合率は下がる
## __これらのトレードオフを確認するために、precision-recall curveがある__

In [ ]:
from sklearn.metrics import precision_recall_curve
ftp,recall,threshold=ftp_recall_curve(
    y_test,
    svc.decision_function(X_test)
)
sns.scatterplot(
    x=ftp[1:],
    y=recall[1:],
    hue=threshold
)
plt.xlabel('precision')
plt.ylabel('recall')

In [ ]:
# データを増やしてカーブを見てみる
X,y=make_blobs(n_samples=(4000,500),cluster_std=[7.0,2],random_state=22)
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=0)
svc=SVC(gamma=.05).fit(X_train,y_train)
precision,recall,threshold = precision_recall_curve(
    y_test,
    svc.decision_function(X_test)
)
close_zero=np.argmin(np.abs(threshold))

plt.scatter(
    precision[close_zero],
    recall[close_zero],
    marker='o',
    # markersize=10,
    # fillstyle=False,
    c='k',
    # mew=2,
    label='threshold zero')
plt.plot(
    precision,
    recall,
    label='precision recall curve')
plt.legend(loc='best')

In [ ]:
rfc = RandomForestClassifier(max_depth=3).fit(X_train,y_train)
rfc.predict_proba(X_test)

In [ ]:
# データを増やしてカーブを見てみる
svc=SVC(gamma=.05).fit(X_train,y_train)
precision_svc,recall_svc,threshold_svc = precision_recall_curve(
    y_test,
    svc.decision_function(X_test)
)
close_zero=np.argmin(np.abs(threshold_svc))

rfc = RandomForestClassifier(n_estimators=100,max_features=2,random_state=0).fit(X_train,y_train)
precision_rfc,recall_rfc,threshold_rfc=precision_recall_curve(
    y_test,
    rfc.predict_proba(X_test)[:,1]
)
close_fifty=np.argmin(np.abs(threshold_rfc-.5))

plt.plot(
    precision_svc,
    recall_svc,
    label='precision recall curve by svc')

plt.plot(
    precision_rfc,
    recall_rfc,
    '--',
    label='precision recall curve by randomforest')

plt.scatter(
    precision_svc[close_zero],
    recall_svc[close_zero],
    marker='o',
    c='k',
    label='threshold zero of svc')

plt.scatter(
    precision_rfc[close_fifty],
    recall_rfc[close_fifty],
    marker='^',
    c='k',
    label='threshold 0.5 of randomforest')

plt.xlabel('precision')
plt.ylabel('recall')
plt.legend(loc='best')

- このカーブを検討するモデルに対して全部見るのはきついので、カーブの下側の面積を平均適合率として取り出して比較する見方もある。

In [ ]:
from sklearn.metrics import average_precision_score
avg_p_scv = average_precision_score(
    y_true=y_test,
    y_score=svc.decision_function(X_test)
)

avg_p_rfc = average_precision_score(
    y_true=y_test,
    y_score=rfc.predict_proba(X_test)[:,1]
)

print(f'avg precision of svc : {avg_p_scv:.2f}\navg precision of random forest : {avg_p_rfc:.2f}')

## もうひとつの不確実性確認　ROCカーブとAUC
- ROC <ins>__R__</ins>eceiver <ins>__O__</ins>perating <ins>__C__</ins>haracteristic 受信者動作特性  
ROCカーブはthresholdを変えた偽陽性率と真陽性率のプロット
$$False Positice Rate = \frac{False Positive}{True Negative \times False Positive} = \frac{本当は陰性なのに陽性にしてしまった}{陰性}$$
$$True Positice Rate = \frac{True Positive}{True Positive \times False negative} = \frac{陽性の予測があっていた}{陽性} = Recall$$


In [ ]:
from sklearn.metrics import roc_curve
fpr,tpr,threshold = roc_curve(
    y_test,
    svc.decision_function(X_test)
)
close_zero=np.argmin(np.abs(threshold))

plt.plot(
    fpr,
    tpr,
    label='ROC svc')

plt.scatter(
    fpr[close_zero],
    tpr[close_zero],
    marker='o',
    c='k',
    label='threshold zero of svc')

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')

- ROCカーブは、左上が理想。偽陽性を低く抑えながら真陽性を高くしたい

In [ ]:
svc=SVC(gamma=.05).fit(X_train,y_train)
ftp_svc,recall_svc,threshold_svc = roc_curve(
    y_test,
    svc.decision_function(X_test)
)
close_zero=np.argmin(np.abs(threshold_svc))

rfc = RandomForestClassifier(n_estimators=100,max_features=2,random_state=0).fit(X_train,y_train)
ftp_rfc,recall_rfc,threshold_rfc=roc_curve(
    y_test,
    rfc.predict_proba(X_test)[:,1]
)
close_fifty=np.argmin(np.abs(threshold_rfc-.5))

plt.plot(
    ftp_svc,
    recall_svc,
    label='ROC curve by svc')

plt.plot(
    ftp_rfc,
    recall_rfc,
    '--',
    label='ROC curve by randomforest')

plt.scatter(
    ftp_svc[close_zero],
    recall_svc[close_zero],
    marker='o',
    c='k',
    label='threshold zero of svc')

plt.scatter(
    ftp_rfc[close_fifty],
    recall_rfc[close_fifty],
    marker='^',
    c='k',
    label='threshold 0.5 of randomforest')

plt.xlabel('False Positiive Rate')
plt.ylabel('True Positive Rate (recall)')
plt.legend(loc='best')

- precision recall curveと同じようにカーブの下側の面積を指標にする
- <ins>A</ins>rea <ins>U</ins>nder the <ins>C</ins>urve = AUC

In [ ]:
from sklearn.metrics import roc_auc_score
auc_scv = roc_auc_score(
    y_true=y_test,
    y_score=svc.decision_function(X_test)
)

auc_rfc = roc_auc_score(
    y_true=y_test,
    y_score=rfc.predict_proba(X_test)[:,1]
)

print(f'auc score of svc : {auc_scv:.2f}\nauc score of random forest : {auc_rfc:.2f}')

In [ ]:
y = digits.target == 9
X_train,X_test,y_train,y_test = train_test_split(
    digits.data,
    y,
    random_state=0
)
for gamma in [1,.05,.01]:
    svc=SVC(gamma=gamma).fit(X_train,y_train)
    accuracy=svc.score(X_test,y_test)
    auc=roc_auc_score(
        y_test,
        svc.decision_function(X_test)
    )
    print(f'gamma = {gamma:.2f}, accuracy = {accuracy:.2f}, auc score = {auc:.2f}')
    fpr,tpr,threshold = roc_curve(
        y_test,
        svc.decision_function(X_test)
    )
    plt.plot(
        fpr,
        tpr,
        label=f'gamma:{gamma}'
    )

plt.xlabel('FPR')
plt.ylabel('TPR')
plt.legend(loc='best')

- ここまでは2クラス分類の話。これを他クラス分類に展開する

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.datasets import load_digits
digits = load_digits()

In [ ]:
import seaborn as sns
sns.set()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    digits.data,
    digits.target,
    random_state=0
)

svc=SVC().fit(X_train,y_train)
matrix=confusion_matrix(
    y_test,
    svc.predict(X_test)
)

In [ ]:
sns.heatmap(
    matrix,
    annot=True,
    # xticklabels='predict',
    # yticklabels='true'
    )
plt.xlabel('predict')
plt.ylabel('true')

In [ ]:
logreg=LogisticRegression(max_iter=2000).fit(X_train,y_train)
matrix=confusion_matrix(
    y_test,
    logreg.predict(X_test)
)
sns.heatmap(
    matrix,
    annot=True,
    cmap='Spectral_r'
    # cmap='rocket_r'
)
plt.xlabel('Predict')
plt.ylabel('True')
print(f'accuracy on test = {logreg.score(X_test,y_test):.2f}')

In [ ]:
from sklearn.metrics import f1_score
print(classification_report(
    y_test,
    logreg.predict(X_test)
))
print(f'micro score {f1_score(y_test,logreg.predict(X_test),average="micro"):.3f}')

- `f1-score`はそれぞれのクラスをpositive,ほかをnegativeの２値分類とした時のprecisionとrecallをもとに計算している。  
- `macro avg`は上記のように算出した`precision`,`recall`,`f1_score`の単純平均値を示す。  
クラス間にサンプルサイズの偏りがあったとしても、`macro avg`は重み付けしない単純な平均  
- `weighted avg`は支度度(support=そのクラスに属するサンプル数)に応じた重み付け平均  
クラス分類では一般的にはこれを見るらしい  
- `micro score`は、支度度ではなく、<ins>全てのクラスの偽陽性、偽陰性、真陽性の総数</ins>を使って算出した値  
ひとつひとつのサンプルを同様に重視する場合はこれを使うらしい・・・

<ins>全てのクラスの偽陽性、偽陰性、真陽性の総数</ins> = モデルがpositiveといったサンプル(偽陽性＋真陽性) もしくは 実際のpositiveサンプル(偽陰性＋真陽性)

## 回帰の場合
- 回帰については基本的に$R^2$でOKとのこと。  
- モデル活用において(何かしらのビジネス決定を行うロジックとして)最小二乗誤差や平均絶対誤差を使う場合は、モデルのチューニングをこれらの誤差が小さくなるように実施すべきこともあるかも、らしい。

## モデルの選択時点でモデルの評価も加味してやりたい場合
- モデルに最適なパラメータチューニングのためのGridSearchCVや、cross_cal_scoreは、accuracyを指標として使っていた
- accuracyは使ったらあかん！という話だったのに...
- sklearnなら、それぞれの引数にあるscoringに、基準となるスコア算出法を明示的に指定すればOK

In [ ]:
print(
    cross_val_score(
        SVC(),
        digits.data,
        digits.target==9,
        scoring='accuracy'
    )
)
print(
    cross_val_score(
        SVC(),
        digits.data,
        digits.target==9,
        scoring='roc_auc'
    )
)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score

X_train, X_test, y_train, y_test = train_test_split(
    digits.data,digits.target == 9,random_state=0
)

def show_result(grid,X_test=X_test,y_test=y_test):
    print(f'best param    = {grid.best_params_}')
    print(f'best score    = {grid.best_score_:.3f}')
    print(f'test accuracy = {grid.score(X_test,y_test):.3f}')
    print(f'test auc      = {roc_auc_score(y_test,grid.decision_function(X_test)):.3f}')

param_grid = {'gamma' : [.0001,.01,.1,1,10]}
grid = GridSearchCV(SVC(), param_grid=param_grid, scoring='accuracy')
grid.fit(X_train,y_train)
print('accuracy')
show_result(grid=grid)

grid = GridSearchCV(SVC(), param_grid=param_grid, scoring='roc_auc')
grid.fit(X_train,y_train)
print('roc_auc')
show_result(grid=grid)

* 使えるスコアの一覧

In [ ]:
from sklearn.metrics._scorer import _SCORERS

def get_(list,num):
    try:
        return f'{num:<3}{list[num]:<30}'
    except:
        return ''

print('使えるスコア一覧')
l = list(_SCORERS.keys())
for i in range(len(l)//3):
    print(
        f'{get_(l,3*i+1)}'
        f'{get_(l,3*i+2)}'
        f'{get_(l,3*i+3)}'
        )

In [ ]:
get_(_SCORERS.keys(),0,1)